In [31]:
import psycopg2
import pandas as pd
import re
from datetime import date

In [32]:
def get_cursor():
    conn = psycopg2.connect(
    database="legislative_testimony",
    user="brianrennie",
    password="")
    cursor = conn.cursor()
    return cursor

cursor = get_cursor()

In [33]:
bill_df = pd.read_csv('../data/all-history-actions.csv', index_col=0)

bill_df

,action,branch,date
id,,,
H1,Referred to the committee on House Ways and Means,House,2021-02-25T16:46:19.933Z
H1,"Reported, in part, by H4000",House,2021-04-15T04:00:00.000Z
H10,Referred to the committee on Public Service,House,2021-04-13T15:59:37.223Z
H10,Senate concurred,Senate,2021-04-13T15:59:37.223Z
H10,Hearing scheduled for 07/28/2021 from 01:00 PM...,Joint,2021-07-22T14:53:10.290Z
...,...,...,...
s2729,Recommended new text for H4126,Senate,2022-02-28T05:00:00.000Z
s2729,Rules suspended,Senate,2022-02-28T05:00:00.000Z
s2729,Substituted as a new text for H4126,Senate,2022-02-28T05:00:00.000Z


In [34]:
bill_df.describe()

,action,branch,date
count,45472,45472,45472
unique,5511,4,13698
top,Senate concurred,House,2021-03-30T03:59:59.000Z
freq,6484,19519,4988


In [35]:
bill_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45472 entries, H1 to s3049
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   action  45472 non-null  object
 1   branch  45472 non-null  object
 2   date    45472 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [27]:
action_counts = bill_df['action'].value_counts().to_dict()

action_counts

enacted_count = bill_df[(bill_df['action'].str.count('(^Reported)')>0)]

enacted_count

# reading = bill_df[(bill_df['action'].str.count('(^Read)')>0)]

# reading

,action,branch,date
id,,,
H1,"Reported, in part, by H4000",House,2021-04-15T04:00:00.000Z
H102,"Reported from the committee on Bonding, Capita...",House,2021-04-12T04:00:00.000Z
H1038,Reported favorably by committee and referred t...,House,2022-05-09T17:19:22.513Z
H1039,Reported favorably by committee and referred t...,House,2022-03-29T15:40:37.450Z
H1040,Reported favorably by committee and referred t...,House,2022-03-29T15:40:39.920Z
...,...,...,...
S35,Reported from the committee on Senate Ways and...,Senate,2021-03-16T04:00:00.000Z
S39,Reported from the committee of conference,Senate,2021-03-25T13:53:09.777Z
S39,Reported on a part of S14,Senate,2021-03-25T13:53:09.777Z


In [37]:
action_counts = bill_df['action'].value_counts().to_dict()

branch_action_count = bill_df['branch'].value_counts()

branch_action_count




House        19519
Senate       17888
Joint         7845
Executive      220
Name: branch, dtype: int64

In [55]:
def number_enacted(year):
    cursor = get_cursor()
    cursor.execute('''SELECT id, action, branch 
                        FROM bill_history
                        WHERE action LIKE '%Reading%'
                      ''')
    result = cursor.fetchall()
    return result

number_enacted(2021)

[('H1917', 'Referred to the committee on Bills in the Third Reading', 'House'),
 ('H3702',
  'Referred to the committee on House Bills in the Third Reading',
  'House'),
 ('H3702', 'Bills in the Third Reading special report accepted', 'House'),
 ('H3722', 'Referred to the committee on Bills in the Third Reading', 'House'),
 ('H3747', 'Referred to the committee on Bills in the Third Reading', 'House'),
 ('H3751', 'Referred to the committee on Bills in the Third Reading', 'House'),
 ('H3762', 'Referred to the committee on Bills in the Third Reading', 'House'),
 ('H3762', 'Bills in the Third Reading Special report accepted', 'House'),
 ('H3762',
  "Governor's amendment considered (in the form approved by the committee on Bills in the Third Reading)",
  'House'),
 ('H3812', 'Referred to the committee on Bills in the Third Reading', 'House'),
 ('H3827', 'Referred to the committee on Bills in the Third Reading', 'House'),
 ('H3827', 'Referred to the committee on Bills in the Third Reading', 